### Conditional Variational Autoencoder(CVAE)
Conditional Variational Autoencoder(CVAE)[^1]是Variational Autoencoder(VAE)[^2]的扩展，在VAE中没有办法对生成的数据加以限制，所以如果在VAE中想生成特定的数据是办不到的。比如在mnist手写数字中，我们想生成特定的数字2，VAE就无能为力了。
因此，CVAE通过对潜层变量和输入数据施加约束，可以生成在某种约束条件下的数据。

在VAE中目标函数如下所示：
$$
\log P ( X ) - D _ { K L } [ Q ( z | X ) \| P ( z | X ) ] = E [ \log P ( X | z ) ] - D _ { K L } [ Q ( z | X ) \| P ( z ) ]
$$
这个目标函数（主要考虑变分下界）要使输入数据经过编码后的潜层变量的分布尽量服从某种先验分布P(Z),并且最小化重建损失。在这个模型中，编码器直接基于输入X来建模潜层变量z，而不考虑潜层输入X的类型（标签），解码器直接基于潜层变量z来重建X，假设得到X1，并没有将要获得那种类型的X1考虑在内。

对VAE进行改进使其可以基于某种约束来生成对应的样本。以mnist手写数字为例，将数字的标签y考虑在内，即编码器为Q(z|X, y), 解码器为P(X|z, y)。上述模型可以写成下面的形式
$$
\log P ( X | y ) - D _ { K L } [ Q ( z | X ,y ) \| P ( z | X ,y ) ] = E [ \log P ( X | z ,y ) ] - D _ { K L } [ Q ( z | X ,y ) \| P ( z | y ) ]
$$
潜层变量z的分布变成了条件概率分布P(z|X,y), 对解码器来说生成的样本也变成了条件概率分布Q(X|z, y)。

### CVAE的实现
本例使用mnist数据集，在VAE的基础上将标签y进行one-hot编码，之后和数据样本进行连接作为输入，在解码时，将潜层变量z和标签y的one-hot编码进行连接，以这种方式实现上述的条件概率分布。
- - -
[^1]: Sohn, Kihyuk, Honglak Lee, and Xinchen Yan. “Learning Structured Output Representation using Deep Conditional Generative Models.” Advances in Neural Information Processing Systems. 2015.

[^2]: Kingma, Diederik P., and Max Welling. "Auto-encoding variational bayes." arXiv preprint arXiv:1312.6114 (2013).

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import os
from sklearn.preprocessing import LabelBinarizer
import numpy as np
from torch.autograd import Variable


project_root = '/Users/JSen/Documents/VAE/'
os.chdir(project_root)

no_cuda = False
cuda_available = not no_cuda and torch.cuda.is_available()

BATCH_SIZE = 64
EPOCH = 100
SEED = 8

torch.manual_seed(SEED)

device = torch.device("cuda" if cuda_available else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda_available else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./MNIST_data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=BATCH_SIZE, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./MNIST_data', train=False, transform=transforms.ToTensor()),
    batch_size=BATCH_SIZE, shuffle=True, **kwargs)

print(train_loader.dataset.train_labels)
print(test_loader.dataset.test_labels)

tensor([ 5,  0,  4,  ...,  5,  6,  8])
tensor([ 7,  2,  1,  ...,  4,  5,  6])


In [2]:
class CVAE(nn.Module):
    def __init__(self):
        super(CVAE, self).__init__()
        
        self.fc1 = nn.Linear(794, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(30, 400)
        self.fc4 = nn.Linear(400, 794)
        
        self.lb = LabelBinarizer()
    #将标签进行one-hot编码
    def to_categrical(self, y: torch.FloatTensor):
        y_n = y.numpy()
        self.lb.fit(list(range(0,10)))
        y_one_hot = self.lb.transform(y_n)
        floatTensor = torch.FloatTensor(y_one_hot)
        return floatTensor
        
    def encode(self, x, y):
        y_c = self.to_categrical(y)
        #输入样本和标签y的one-hot向量连接
        con = torch.cat((x, y_c), 1)
        h1 = F.relu(self.fc1(con))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        #训练时使用重参数化技巧，测试时不用。（测试时应该可以用）
        if self.training:
            std = torch.exp(0.5*logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z, y):
        y_c = self.to_categrical(y)
        #解码器的输入：将z和y的one-hot向量连接
        cat = torch.cat((z, y_c), 1)
        h3 = F.relu(self.fc3(cat))
        return F.sigmoid(self.fc4(h3))

    def forward(self, x, y):
        mu, logvar = self.encode(x.view(-1, 784), y)
        z = self.reparameterize(mu, logvar)
        return self.decode(z, y), mu, logvar

# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, size_average=False)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

model = CVAE().to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [3]:
def train(epoch):
    #Sets the module in training mode.
    model.train()
    train_loss = 0

#     batch_idx, (data, label) =enumerate(train_loader).__next__()
    for batch_idx, (data, label) in enumerate(train_loader):
        data = data.to(device) #[64, 1, 28, 28]
        
        optimizer.zero_grad()
        
        recon_batch, mu, logvar = model(data, label)
#         print(recon_batch.shape) #[64, 794]
        #训练样本展平，在每个样本后面连接标签的one-hot向量
        flat_data = data.view(-1, data.shape[2]*data.shape[3])
#         print(data.shape, flat_data.shape)
        y_condition = model.to_categrical(label)
        con = torch.cat((flat_data, y_condition), 1)
        
        loss = loss_function(recon_batch, con, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()

        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(data),
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))





In [4]:
index, (data, label) =enumerate(train_loader).__next__()
print(index, data.shape, label.shape)

0 torch.Size([64, 1, 28, 28]) torch.Size([64])


In [5]:
def test(epoch):
    #Sets the module in evaluation mode
    model.eval()
    test_loss = 0

    with torch.no_grad():
        for i, (data, label) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data, label)
            
            flat_data = data.view(-1, data.shape[2]*data.shape[3])
            
            y_condition = model.to_categrical(label)
            con = torch.cat((flat_data, y_condition), 1)
            test_loss += loss_function(recon_batch, con, mu, logvar).item()

            if i == 0:
                n = min(data.size(0), 8)
                recon_image = recon_batch[:, 0:recon_batch.shape[1]-10]
                print(recon_image.shape)
                recon_image = recon_image.view(BATCH_SIZE, 1, 28,28)
                print('---',recon_image.shape)
                comparison = torch.cat([data[:n],
                                      recon_image.view(BATCH_SIZE, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


for epoch in range(1, EPOCH + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        #采样过程
        sample = torch.randn(64, 20).to(device)
      
        c = np.zeros(shape=(sample.shape[0],))
        rand = np.random.randint(0, 10)
        print(f"Random number: {rand}")
        c[:] = rand
        c = torch.FloatTensor(c)
        sample = model.decode(sample, c).cpu()
        #模型的输出矩阵：每一行的末尾都加了one-hot向量，要去掉这个one-hot向量再转换为图片。
        generated_image = sample[:, 0:sample.shape[1]-10]
        
        
        save_image(generated_image.view(64, 1, 28, 28),
                   'results/sample_' + str(epoch) + '.png')



Train Epoch: 1 [0/60000 (0%)]	Loss: 557.402771
Train Epoch: 1 [640/60000 (1%)]	Loss: 304.169098
Train Epoch: 1 [1280/60000 (2%)]	Loss: 245.004852
Train Epoch: 1 [1920/60000 (3%)]	Loss: 226.045837
Train Epoch: 1 [2560/60000 (4%)]	Loss: 213.904572
Train Epoch: 1 [3200/60000 (5%)]	Loss: 216.395126
Train Epoch: 1 [3840/60000 (6%)]	Loss: 204.337250
Train Epoch: 1 [4480/60000 (7%)]	Loss: 204.937653
Train Epoch: 1 [5120/60000 (9%)]	Loss: 190.433853
Train Epoch: 1 [5760/60000 (10%)]	Loss: 192.318848
Train Epoch: 1 [6400/60000 (11%)]	Loss: 191.780121
Train Epoch: 1 [7040/60000 (12%)]	Loss: 186.929581
Train Epoch: 1 [7680/60000 (13%)]	Loss: 164.364624
Train Epoch: 1 [8320/60000 (14%)]	Loss: 167.252563
Train Epoch: 1 [8960/60000 (15%)]	Loss: 159.597061
Train Epoch: 1 [9600/60000 (16%)]	Loss: 163.007812
Train Epoch: 1 [10240/60000 (17%)]	Loss: 156.970749
Train Epoch: 1 [10880/60000 (18%)]	Loss: 160.360352
Train Epoch: 1 [11520/60000 (19%)]	Loss: 164.100754
Train Epoch: 1 [12160/60000 (20%)]	Loss: 

Train Epoch: 2 [39680/60000 (66%)]	Loss: 113.531502
Train Epoch: 2 [40320/60000 (67%)]	Loss: 106.705704
Train Epoch: 2 [40960/60000 (68%)]	Loss: 107.504150
Train Epoch: 2 [41600/60000 (69%)]	Loss: 113.620697
Train Epoch: 2 [42240/60000 (70%)]	Loss: 113.400223
Train Epoch: 2 [42880/60000 (71%)]	Loss: 110.348228
Train Epoch: 2 [43520/60000 (72%)]	Loss: 109.928001
Train Epoch: 2 [44160/60000 (74%)]	Loss: 106.934937
Train Epoch: 2 [44800/60000 (75%)]	Loss: 116.209610
Train Epoch: 2 [45440/60000 (76%)]	Loss: 115.114342
Train Epoch: 2 [46080/60000 (77%)]	Loss: 112.993523
Train Epoch: 2 [46720/60000 (78%)]	Loss: 112.090202
Train Epoch: 2 [47360/60000 (79%)]	Loss: 110.604111
Train Epoch: 2 [48000/60000 (80%)]	Loss: 111.704346
Train Epoch: 2 [48640/60000 (81%)]	Loss: 104.812492
Train Epoch: 2 [49280/60000 (82%)]	Loss: 109.758942
Train Epoch: 2 [49920/60000 (83%)]	Loss: 111.573532
Train Epoch: 2 [50560/60000 (84%)]	Loss: 107.026810
Train Epoch: 2 [51200/60000 (85%)]	Loss: 113.409065
Train Epoch:

Train Epoch: 4 [17920/60000 (30%)]	Loss: 107.803215
Train Epoch: 4 [18560/60000 (31%)]	Loss: 107.439041
Train Epoch: 4 [19200/60000 (32%)]	Loss: 105.700264
Train Epoch: 4 [19840/60000 (33%)]	Loss: 113.495041
Train Epoch: 4 [20480/60000 (34%)]	Loss: 106.434830
Train Epoch: 4 [21120/60000 (35%)]	Loss: 114.316612
Train Epoch: 4 [21760/60000 (36%)]	Loss: 110.213669
Train Epoch: 4 [22400/60000 (37%)]	Loss: 104.473854
Train Epoch: 4 [23040/60000 (38%)]	Loss: 101.837769
Train Epoch: 4 [23680/60000 (39%)]	Loss: 106.611275
Train Epoch: 4 [24320/60000 (41%)]	Loss: 117.717865
Train Epoch: 4 [24960/60000 (42%)]	Loss: 108.616837
Train Epoch: 4 [25600/60000 (43%)]	Loss: 105.154068
Train Epoch: 4 [26240/60000 (44%)]	Loss: 106.883247
Train Epoch: 4 [26880/60000 (45%)]	Loss: 107.286011
Train Epoch: 4 [27520/60000 (46%)]	Loss: 109.384369
Train Epoch: 4 [28160/60000 (47%)]	Loss: 109.321457
Train Epoch: 4 [28800/60000 (48%)]	Loss: 112.644119
Train Epoch: 4 [29440/60000 (49%)]	Loss: 109.603554
Train Epoch:

Train Epoch: 5 [57600/60000 (96%)]	Loss: 98.905472
Train Epoch: 5 [58240/60000 (97%)]	Loss: 103.895004
Train Epoch: 5 [58880/60000 (98%)]	Loss: 111.692032
Train Epoch: 5 [59520/60000 (99%)]	Loss: 115.047897
====> Epoch: 5 Average loss: 104.7388
torch.Size([64, 784])
--- torch.Size([64, 1, 28, 28])
====> Test set loss: 95.2347
Random number: 7
Train Epoch: 6 [0/60000 (0%)]	Loss: 103.965927
Train Epoch: 6 [640/60000 (1%)]	Loss: 105.174377
Train Epoch: 6 [1280/60000 (2%)]	Loss: 102.858261
Train Epoch: 6 [1920/60000 (3%)]	Loss: 105.717110
Train Epoch: 6 [2560/60000 (4%)]	Loss: 105.198830
Train Epoch: 6 [3200/60000 (5%)]	Loss: 107.547981
Train Epoch: 6 [3840/60000 (6%)]	Loss: 106.040871
Train Epoch: 6 [4480/60000 (7%)]	Loss: 105.150047
Train Epoch: 6 [5120/60000 (9%)]	Loss: 104.991096
Train Epoch: 6 [5760/60000 (10%)]	Loss: 102.587502
Train Epoch: 6 [6400/60000 (11%)]	Loss: 102.241173
Train Epoch: 6 [7040/60000 (12%)]	Loss: 105.877876
Train Epoch: 6 [7680/60000 (13%)]	Loss: 99.085785
Train 

Train Epoch: 7 [35840/60000 (60%)]	Loss: 103.954742
Train Epoch: 7 [36480/60000 (61%)]	Loss: 101.991959
Train Epoch: 7 [37120/60000 (62%)]	Loss: 102.053680
Train Epoch: 7 [37760/60000 (63%)]	Loss: 102.235512
Train Epoch: 7 [38400/60000 (64%)]	Loss: 100.542320
Train Epoch: 7 [39040/60000 (65%)]	Loss: 98.203133
Train Epoch: 7 [39680/60000 (66%)]	Loss: 108.458801
Train Epoch: 7 [40320/60000 (67%)]	Loss: 97.811073
Train Epoch: 7 [40960/60000 (68%)]	Loss: 101.747467
Train Epoch: 7 [41600/60000 (69%)]	Loss: 103.586060
Train Epoch: 7 [42240/60000 (70%)]	Loss: 103.935150
Train Epoch: 7 [42880/60000 (71%)]	Loss: 101.957016
Train Epoch: 7 [43520/60000 (72%)]	Loss: 105.259270
Train Epoch: 7 [44160/60000 (74%)]	Loss: 94.209824
Train Epoch: 7 [44800/60000 (75%)]	Loss: 109.468788
Train Epoch: 7 [45440/60000 (76%)]	Loss: 106.013191
Train Epoch: 7 [46080/60000 (77%)]	Loss: 103.365921
Train Epoch: 7 [46720/60000 (78%)]	Loss: 107.109482
Train Epoch: 7 [47360/60000 (79%)]	Loss: 104.871445
Train Epoch: 7 

...

Train Epoch: 94 [26880/60000 (45%)]	Loss: 94.679779
Train Epoch: 94 [27520/60000 (46%)]	Loss: 97.569412
Train Epoch: 94 [28160/60000 (47%)]	Loss: 98.699844
Train Epoch: 94 [28800/60000 (48%)]	Loss: 101.165062
Train Epoch: 94 [29440/60000 (49%)]	Loss: 93.309029
Train Epoch: 94 [30080/60000 (50%)]	Loss: 94.619484
Train Epoch: 94 [30720/60000 (51%)]	Loss: 96.301323
Train Epoch: 94 [31360/60000 (52%)]	Loss: 100.560326
Train Epoch: 94 [32000/60000 (53%)]	Loss: 97.999420
Train Epoch: 94 [32640/60000 (54%)]	Loss: 92.571823
Train Epoch: 94 [33280/60000 (55%)]	Loss: 98.744087
Train Epoch: 94 [33920/60000 (57%)]	Loss: 100.480072
Train Epoch: 94 [34560/60000 (58%)]	Loss: 98.260376
Train Epoch: 94 [35200/60000 (59%)]	Loss: 92.112152
Train Epoch: 94 [35840/60000 (60%)]	Loss: 91.550949
Train Epoch: 94 [36480/60000 (61%)]	Loss: 98.308052
Train Epoch: 94 [37120/60000 (62%)]	Loss: 97.668777
Train Epoch: 94 [37760/60000 (63%)]	Loss: 99.823524
Train Epoch: 94 [38400/60000 (64%)]	Loss: 90.320419
Train Epo

Train Epoch: 96 [4480/60000 (7%)]	Loss: 95.830200
Train Epoch: 96 [5120/60000 (9%)]	Loss: 97.471039
Train Epoch: 96 [5760/60000 (10%)]	Loss: 97.559433
Train Epoch: 96 [6400/60000 (11%)]	Loss: 95.950241
Train Epoch: 96 [7040/60000 (12%)]	Loss: 96.426491
Train Epoch: 96 [7680/60000 (13%)]	Loss: 96.527443
Train Epoch: 96 [8320/60000 (14%)]	Loss: 98.376472
Train Epoch: 96 [8960/60000 (15%)]	Loss: 96.844337
Train Epoch: 96 [9600/60000 (16%)]	Loss: 94.224815
Train Epoch: 96 [10240/60000 (17%)]	Loss: 94.065353
Train Epoch: 96 [10880/60000 (18%)]	Loss: 92.924187
Train Epoch: 96 [11520/60000 (19%)]	Loss: 98.009232
Train Epoch: 96 [12160/60000 (20%)]	Loss: 95.422737
Train Epoch: 96 [12800/60000 (21%)]	Loss: 97.572479
Train Epoch: 96 [13440/60000 (22%)]	Loss: 99.480026
Train Epoch: 96 [14080/60000 (23%)]	Loss: 97.190102
Train Epoch: 96 [14720/60000 (25%)]	Loss: 91.969894
Train Epoch: 96 [15360/60000 (26%)]	Loss: 97.230812
Train Epoch: 96 [16000/60000 (27%)]	Loss: 99.348427
Train Epoch: 96 [16640/

Train Epoch: 97 [44160/60000 (74%)]	Loss: 94.166260
Train Epoch: 97 [44800/60000 (75%)]	Loss: 98.256889
Train Epoch: 97 [45440/60000 (76%)]	Loss: 96.288696
Train Epoch: 97 [46080/60000 (77%)]	Loss: 95.474045
Train Epoch: 97 [46720/60000 (78%)]	Loss: 96.780434
Train Epoch: 97 [47360/60000 (79%)]	Loss: 91.534485
Train Epoch: 97 [48000/60000 (80%)]	Loss: 97.179245
Train Epoch: 97 [48640/60000 (81%)]	Loss: 95.743454
Train Epoch: 97 [49280/60000 (82%)]	Loss: 92.452774
Train Epoch: 97 [49920/60000 (83%)]	Loss: 96.687027
Train Epoch: 97 [50560/60000 (84%)]	Loss: 94.124825
Train Epoch: 97 [51200/60000 (85%)]	Loss: 98.376801
Train Epoch: 97 [51840/60000 (86%)]	Loss: 94.497597
Train Epoch: 97 [52480/60000 (87%)]	Loss: 98.265175
Train Epoch: 97 [53120/60000 (88%)]	Loss: 96.154549
Train Epoch: 97 [53760/60000 (90%)]	Loss: 94.394379
Train Epoch: 97 [54400/60000 (91%)]	Loss: 94.469994
Train Epoch: 97 [55040/60000 (92%)]	Loss: 94.945107
Train Epoch: 97 [55680/60000 (93%)]	Loss: 93.668861
Train Epoch:

Train Epoch: 99 [22400/60000 (37%)]	Loss: 95.682114
Train Epoch: 99 [23040/60000 (38%)]	Loss: 95.262321
Train Epoch: 99 [23680/60000 (39%)]	Loss: 98.037918
Train Epoch: 99 [24320/60000 (41%)]	Loss: 94.353188
Train Epoch: 99 [24960/60000 (42%)]	Loss: 95.258751
Train Epoch: 99 [25600/60000 (43%)]	Loss: 97.917427
Train Epoch: 99 [26240/60000 (44%)]	Loss: 96.639191
Train Epoch: 99 [26880/60000 (45%)]	Loss: 94.600082
Train Epoch: 99 [27520/60000 (46%)]	Loss: 95.343170
Train Epoch: 99 [28160/60000 (47%)]	Loss: 91.657921
Train Epoch: 99 [28800/60000 (48%)]	Loss: 97.006439
Train Epoch: 99 [29440/60000 (49%)]	Loss: 95.606247
Train Epoch: 99 [30080/60000 (50%)]	Loss: 98.472885
Train Epoch: 99 [30720/60000 (51%)]	Loss: 93.821274
Train Epoch: 99 [31360/60000 (52%)]	Loss: 96.787315
Train Epoch: 99 [32000/60000 (53%)]	Loss: 98.822998
Train Epoch: 99 [32640/60000 (54%)]	Loss: 97.496490
Train Epoch: 99 [33280/60000 (55%)]	Loss: 94.069710
Train Epoch: 99 [33920/60000 (57%)]	Loss: 92.305786
Train Epoch:

====> Test set loss: 88.7073
Random number: 8
